In [3]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs

%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return


In [74]:
def yahoo_analysts(ticker):
    yahoo_url = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'.format(ticker)
    res = requests.get(yahoo_url)
    soup = bs(requests.get(yahoo_url).text, "lxml")
    table = soup.find_all('table')

    # Earnings Estimates
    raw_html_table = table[0]

    i = 0
    headers = ['Current Qtr', 'Next Qtr', 'Current Year', 'Next Year']
    idx = ['No. of Analysts', 'Avg. Estimate', 'Low Estimate', 'High Estimate', 'Year Ago EPS']
    earnings_df = pd.DataFrame(columns = headers, index = idx)
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        if i != 0:
            split_row = individual_row[0].split('<!-- /react-text -->')
            col = 0
            for j in split_row:
                if '-->' in j:
                    earnings_df.iloc[i-1,col] = j.split('-->')[-1]

                    col += 1

        i += 1

    # Revenue Estimate
    raw_html_table = table[1]

    i = 0
    idx = ['No. of Analysts', 'Avg. Estimate', 'Low Estimate', 'High Estimate', 
           'Year Ago Sales', 'Sales Growth (year/est)']
    revenue_df = pd.DataFrame(columns = headers, index = idx)
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        if i != 0:
            split_row = individual_row[0].split('<!-- /react-text -->')
            col = 0
            for j in split_row:
                if '-->' in j:
                    revenue_df.iloc[i-1,col] = j.split('-->')[-1]

                    col += 1

        i += 1

    # Earnings History
    raw_html_table = table[2]

    i = 0
    idx = ['EPS Est.', 'EPS Actual', 'Difference', 'Surprise %']
    earnings_history = pd.DataFrame(columns = headers, index = idx)
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        if i != 0:
            split_row = individual_row[0].split('<!-- /react-text -->')
            col = 0
            for j in split_row:
                if '-->' in j:
                    earnings_history.iloc[i-1,col] = j.split('-->')[-1]

                    col += 1

        i += 1

    # EPS Trend
    raw_html_table = table[3]

    i = 0
    idx = ['Current Estimate', '7 Days Ago', '30 Days Ago',
           '60 Days Ago', '90 Days Ago']
    eps_trend = pd.DataFrame(columns = headers, index = idx)
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        if i != 0:
            split_row = individual_row[0].split('<!-- /react-text -->')
            col = 0
            for j in split_row:
                if '-->' in j:
                    eps_trend.iloc[i-1,col] = j.split('-->')[-1]

                    col += 1

        i += 1

    # EPS Revisions
    raw_html_table = table[4]

    i = 0
    idx = ['Up last 7 days', 'Up last 30 days', 'Down last 7 days',
           'Up last 30 days']
    eps_revis = pd.DataFrame(columns = headers, index = idx)
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        if i != 0:
            split_row = individual_row[0].split('<!-- /react-text -->')
            col = 0
            for j in split_row:
                if '-->' in j:
                    eps_revis.iloc[i-1,col] = j.split('-->')[-1]

                    col += 1

        i += 1
    
    return [earnings_df, revenue_df, earnings_history, eps_trend, eps_revis]

In [77]:
test = yahoo_analysts('MSFT')
for tbl in test:
    print(tbl)

                Current Qtr Next Qtr Current Year Next Year
No. of Analysts          30       30           34        34
Avg. Estimate          0.85        1         3.65      3.92
Low Estimate           0.79     0.85          3.4      3.59
High Estimate          0.91     1.06         3.76      4.53
Year Ago EPS            0.7     1.06         3.29      3.65
                        Current Qtr Next Qtr Current Year Next Year
No. of Analysts                  28       28           33        33
Avg. Estimate                25.77B      28B      107.25B   116.42B
Low Estimate                 25.58B   26.82B      105.85B    109.5B
High Estimate                26.35B   28.79B      108.34B    121.5B
Year Ago Sales               23.56B    24.7B       96.66B   107.25B
Sales Growth (year/est)       9.40%   13.40%       11.00%     8.60%
           Current Qtr Next Qtr Current Year Next Year
EPS Est.           0.7     0.71         0.72      0.87
EPS Actual         0.7     1.06         0.84      0.96